In [84]:
import xgboost as xgb

In [13]:
#from sklearn.model_selection import train_test_split

In [85]:
import pandas as pd
import numpy as np

In [86]:
import matplotlib.pyplot as plt
import seaborn as SNS
%matplotlib inline

In [210]:
from sklearn.preprocessing import StandardScaler

In [87]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [615]:
model = xgb.XGBRegressor(n_estimators=1000, colsample_bytree =0.3,
 gamma = 0.0,
 learning_rate =  0.15,
 max_depth = 5,
 min_child_weight = 1)

In [335]:
#get covid restriction index
covid_data = pd.read_csv('data\owid-covid-data.csv')
covid_data = covid_data[covid_data['iso_code'] == 'CZE']
restriction_index = pd.DataFrame()
restriction_index = covid_data[['date','stringency_index']]
restriction_index = restriction_index.rename(columns = {'date':'ds'})
restriction_index['ds']=  pd.to_datetime(restriction_index['ds'])
restriction_index.head()

,ds,stringency_index
49132,2020-03-01,19.44
49133,2020-03-02,19.44
49134,2020-03-03,19.44
49135,2020-03-04,19.44
49136,2020-03-05,19.44


In [584]:
df = pd.read_csv('prophetdf.csv')
df['ds']=  pd.to_datetime(df['ds'])

In [509]:
forecast.head()

,Unnamed: 0,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,1. svátek vánoční,1. svátek vánoční_lower,1. svátek vánoční_upper,...,yearly,yearly_lower,yearly_upper,Štědrý den,Štědrý den_lower,Štědrý den_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,0,2019-07-02,42910.668771,6310.605593,54678.287934,42910.668771,42910.668771,0.0,0.0,0.0,...,-0.319278,-0.319278,-0.319278,0.0,0.0,0.0,0.0,0.0,0.0,30871.938406
1,1,2019-07-03,42941.568091,3603.090208,53020.134368,42941.568091,42941.568091,0.0,0.0,0.0,...,-0.316820,-0.316820,-0.316820,0.0,0.0,0.0,0.0,0.0,0.0,29545.852006
2,2,2019-07-04,42972.467412,3080.075047,54489.186101,42972.467412,42972.467412,0.0,0.0,0.0,...,-0.313487,-0.313487,-0.313487,0.0,0.0,0.0,0.0,0.0,0.0,27928.754660
3,3,2019-07-05,43003.366732,2042.728777,49271.745488,43003.366732,43003.366732,0.0,0.0,0.0,...,-0.309357,-0.309357,-0.309357,0.0,0.0,0.0,0.0,0.0,0.0,25372.628485
4,4,2019-07-06,43034.266053,-1590.290740,46652.295447,43034.266053,43034.266053,0.0,0.0,0.0,...,-0.304512,-0.304512,-0.304512,0.0,0.0,0.0,0.0,0.0,0.0,23177.983567


In [475]:
#feats month, dayofyear, week, year, holiday_season(nov 1 - dec 23), holiday, covid_lockdown, weekday

df['Day'] = df['ds'].dt.day
df['Week'] = df['ds'].dt.week
df['Month'] = df['ds'].dt.month
df['Year'] = df['ds'].dt.year
df['DayofYear'] = df['ds'].dt.dayofyear
df['DayofWeek'] = df['ds'].dt.dayofweek

C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\619729888.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['Week'] = df['ds'].dt.week


In [585]:
#join stringency index to full dataset, fill nan with zero
data = pd.merge(df,restriction_index,on='ds',how='left')
data = data.fillna(0)

In [339]:
forecast.columns.values

array(['ds', 'trend', 'yhat', 'weekly', 'yearly', 'holidays'],
      dtype=object)

In [586]:
#prophet prediction
forecast = pd.read_csv('prophet_forecast.csv')
forecast.columns.values

array(['Unnamed: 0', 'ds', 'trend', 'yhat_lower', 'yhat_upper',
       'trend_lower', 'trend_upper', '1. svátek vánoční',
       '1. svátek vánoční_lower', '1. svátek vánoční_upper',
       '2. svátek vánoční', '2. svátek vánoční_lower',
       '2. svátek vánoční_upper', 'Den boje za svobodu a demokracii',
       'Den boje za svobodu a demokracii_lower',
       'Den boje za svobodu a demokracii_upper',
       'Den obnovy samostatného českého státu',
       'Den obnovy samostatného českého státu_lower',
       'Den obnovy samostatného českého státu_upper',
       'Den slovanských věrozvěstů Cyrila a Metoděje',
       'Den slovanských věrozvěstů Cyrila a Metoděje_lower',
       'Den slovanských věrozvěstů Cyrila a Metoděje_upper',
       'Den upálení mistra Jana Husa',
       'Den upálení mistra Jana Husa_lower',
       'Den upálení mistra Jana Husa_upper',
       'Den vzniku samostatného československého státu',
       'Den vzniku samostatného československého státu_lower',
       'Den 

In [587]:
forecast = forecast[['ds', 'trend','yhat','weekly', 'yearly','holidays','yhat_lower', 'yhat_upper']]
forecast.head()

,ds,trend,yhat,weekly,yearly,holidays,yhat_lower,yhat_upper
0,2019-07-02,42910.668771,30871.938406,0.038724,-0.319278,0.000000,6310.605593,54678.287934
1,2019-07-03,42941.568091,29545.852006,0.004868,-0.316820,0.000000,3603.090208,53020.134368
2,2019-07-04,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,3080.075047,54489.186101
3,2019-07-05,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,2042.728777,49271.745488
4,2019-07-06,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,-1590.290740,46652.295447


In [252]:
forecast['weekly'].value_counts()

-0.056505    2
-0.036591    2
 0.038724    1
 0.117664    1
-0.036591    1
            ..
 0.074937    1
 0.117664    1
 0.038724    1
 0.004868    1
-0.036591    1
Name: weekly, Length: 921, dtype: int64

In [588]:
forecast['ds']=  pd.to_datetime(forecast['ds'])
data = pd.merge(data,forecast,on='ds',how='left')
data

,ds,y,stringency_index,trend,yhat,weekly,yearly,holidays,yhat_lower,yhat_upper
0,2019-06-01,4087.236534,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-06-02,65.848327,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-06-03,1364.979529,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-06-04,68385.521621,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-06-05,65135.657001,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
953,2022-01-10,107064.467839,37.75,113715.142266,124649.179829,0.117664,-0.021511,0.0,99583.468594,147752.362978
954,2022-01-11,110698.874419,37.75,113813.939173,117475.785954,0.038724,-0.006550,0.0,92289.704472,142821.215328
955,2022-01-12,100503.117963,37.74,113912.736079,115301.625990,0.004868,0.007325,0.0,89936.061247,139443.771693
956,2022-01-13,97576.039846,37.73,114011.532986,112108.702382,-0.036591,0.019901,0.0,87727.305861,135896.927051


In [589]:
#remove outliers and first month to match forecast df
data = data.drop(index=[366,468,774,314,957]) #180, 213


In [159]:
data.head()

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays
0,2019-06-01,4087.236534,1,22,6,2019,152,5,0.0,NaN,NaN,NaN,NaN,NaN
1,2019-06-02,65.848327,2,22,6,2019,153,6,0.0,NaN,NaN,NaN,NaN,NaN
2,2019-06-03,1364.979529,3,23,6,2019,154,0,0.0,NaN,NaN,NaN,NaN,NaN
3,2019-06-04,68385.521621,4,23,6,2019,155,1,0.0,NaN,NaN,NaN,NaN,NaN
4,2019-06-05,65135.657001,5,23,6,2019,156,2,0.0,NaN,NaN,NaN,NaN,NaN


In [590]:
#data['Day'] = data['ds'].dt.day
data['Week'] = data['ds'].dt.week
data['Month'] = data['ds'].dt.month
data['Year'] = data['ds'].dt.year
data['DayofYear'] = data['ds'].dt.dayofyear
data['DayofWeek'] = data['ds'].dt.dayofweek

C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\3732891750.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['Week'] = data['ds'].dt.week


In [591]:
#holiday_season(nov 1 - dec 23)
def holiday_season(x):
    if 305 <= x <= 357:
        return 1
    return 0
data['holiday_season'] = data['DayofYear'] .apply(holiday_season)

In [257]:
data.head()
#onehot encode DayofWeek

,ds,y,Day,Week,Month,Year,DayofYear,DayofWeek,stringency_index,trend,yhat,weekly,yearly,holidays,holiday_season
0,2019-06-01,4087.236534,1,22,6,2019,152,5,0.0,NaN,NaN,NaN,NaN,NaN,0
1,2019-06-02,65.848327,2,22,6,2019,153,6,0.0,NaN,NaN,NaN,NaN,NaN,0
2,2019-06-03,1364.979529,3,23,6,2019,154,0,0.0,NaN,NaN,NaN,NaN,NaN,0
3,2019-06-04,68385.521621,4,23,6,2019,155,1,0.0,NaN,NaN,NaN,NaN,NaN,0
4,2019-06-05,65135.657001,5,23,6,2019,156,2,0.0,NaN,NaN,NaN,NaN,NaN,0


In [592]:
#offset
data['y1'] = data['y'].shift(1)

In [593]:
weather = pd.read_csv('weather.csv')

In [594]:
weather['ds']=  pd.to_datetime(weather['ds'])
data = pd.merge(data,weather,on='ds',how='left')

In [595]:
data = data[30:]
data

,ds,y,stringency_index,trend,yhat,weekly,yearly,holidays,yhat_lower,yhat_upper,...,Month,Year,DayofYear,DayofWeek,holiday_season,y1,temperature,condition,humidity,condition_severity
30,2019-07-02,29825.892188,0.00,42910.668771,30871.938406,0.038724,-0.319278,0.000000,6310.605593,54678.287934,...,7,2019,183,1,0,30217.879866,21.1,Partly cloudy,63.0,0
31,2019-07-03,30148.139918,0.00,42941.568091,29545.852006,0.004868,-0.316820,0.000000,3603.090208,53020.134368,...,7,2019,184,2,0,29825.892188,18.8,Sunny,51.0,0
32,2019-07-04,28886.698448,0.00,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,3080.075047,54489.186101,...,7,2019,185,3,0,30148.139918,20.4,Sunny,54.0,0
33,2019-07-05,29277.998737,0.00,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,2042.728777,49271.745488,...,7,2019,186,4,0,28886.698448,22.0,Partly cloudy,51.0,0
34,2019-07-06,21320.833411,0.00,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,-1590.290740,46652.295447,...,7,2019,187,5,0,29277.998737,25.0,Sunny,56.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,2022-01-09,101228.700023,37.76,113616.345360,117891.200706,0.074937,-0.037312,0.000000,93709.816730,142479.266634,...,1,2022,9,6,0,92744.263348,2.7,Overcast,82.0,0
949,2022-01-10,107064.467839,37.75,113715.142266,124649.179829,0.117664,-0.021511,0.000000,99583.468594,147752.362978,...,1,2022,10,0,0,101228.700023,0.1,Overcast,86.0,0
950,2022-01-11,110698.874419,37.75,113813.939173,117475.785954,0.038724,-0.006550,0.000000,92289.704472,142821.215328,...,1,2022,11,1,0,107064.467839,-1.2,Sunny,85.0,0
951,2022-01-12,100503.117963,37.74,113912.736079,115301.625990,0.004868,0.007325,0.000000,89936.061247,139443.771693,...,1,2022,12,2,0,110698.874419,-1.6,Partly cloudy,76.0,0


In [596]:
#set date to index
data = data.reset_index(drop=True)

In [597]:
X = data.drop(columns = ['ds','y', 'condition', 'stringency_index','condition_severity'])
y =  data['y']

In [598]:
X_train = X[:-7]
X_test = X[-7:]
y_train = y[:-7]
y_test = y[-7:]

In [212]:
#scaler= StandardScaler()

In [391]:
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)

In [423]:
X_test.iloc[2]

Day                    9.000000
Week                   1.000000
Month                  1.000000
Year                2022.000000
DayofYear              9.000000
DayofWeek              6.000000
trend             113616.345360
yhat              117891.200706
weekly                 0.074937
yearly                -0.037312
holidays               0.000000
holiday_season         0.000000
y1                 92744.263348
temperature            2.700000
humidity              82.000000
Name: 918, dtype: float64

In [525]:
X_train

,trend,yhat,weekly,yearly,holidays,Week,Month,Year,DayofYear,DayofWeek,holiday_season,temperature,humidity
0,42910.668771,30871.938406,0.038724,-0.319278,0.000000,27,7,2019,183,1,0,21.1,63.0
1,42941.568091,29545.852006,0.004868,-0.316820,0.000000,27,7,2019,184,2,0,18.8,51.0
2,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,27,7,2019,185,3,0,20.4,54.0
3,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,27,7,2019,186,4,0,22.0,51.0
4,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,27,7,2019,187,5,0,25.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,110652.438165,141747.271991,-0.056505,0.337519,0.000000,49,12,2021,344,4,1,-0.7,95.0
889,110751.235072,127595.763459,-0.143097,0.295190,0.000000,49,12,2021,345,5,1,1.1,93.0
890,110850.031978,147116.847023,0.074937,0.252233,0.000000,49,12,2021,346,6,1,0.0,89.0
891,110948.828885,147211.230378,0.117664,0.209175,0.000000,50,12,2021,347,0,1,1.9,97.0


In [616]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [528]:
#y_pred= model.predict(X_test)

In [617]:
y_pred = []
for x in range(len(X_test)):
    test = X_test[x:x+1]
    #test = scaler.transform(test)
    pred = model.predict(test)
    X_test['y1'][x+len(X_train)] = float(pred) 
    y_pred.append(float(pred))


C:\Users\Alex\AppData\Local\Temp\ipykernel_447732\2990610742.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['y1'][x+len(X_train)] = float(pred)


In [581]:
y_pred

[83810.7734375,
 67658.4921875,
 141855.796875,
 132872.703125,
 88844.6640625,
 89580.734375,
 98459.6328125]

In [618]:
np.sqrt(mean_squared_error(y_test,y_pred))

12056.76412452032

In [504]:
X_test

,Week,Month,Year,DayofYear,DayofWeek,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,...,Štědrý den_lower,Štědrý den_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat,holiday_season,y1,temperature,humidity
923,50,12,2021,349,2,111146.422698,100880.169443,149503.471304,111146.422698,111146.422698,...,0.000000,0.000000,0.0,0.0,0.0,125559.711120,1,183912.313620,5.8,91.0
924,50,12,2021,350,3,111245.219604,92168.428938,142392.059159,111245.209573,111245.777527,...,0.000000,0.000000,0.0,0.0,0.0,116572.737933,1,184233.647435,4.9,88.0
925,50,12,2021,351,4,111344.016511,85565.012147,135878.704840,111342.850435,111347.099689,...,0.000000,0.000000,0.0,0.0,0.0,110173.161376,1,138778.306261,3.5,87.0
926,50,12,2021,352,5,111442.813417,72869.656505,121451.569208,111439.476118,111447.962186,...,0.000000,0.000000,0.0,0.0,0.0,96581.426725,1,109987.895659,1.9,80.0
927,50,12,2021,353,6,111541.610324,93537.543198,141153.123323,111535.778211,111549.516331,...,0.000000,0.000000,0.0,0.0,0.0,117234.523304,1,89303.723054,6.0,81.0
928,51,12,2021,354,0,111640.407230,93874.804881,141021.955444,111632.640456,111652.766905,...,0.000000,0.000000,0.0,0.0,0.0,118678.725333,1,95343.122476,0.8,77.0
929,51,12,2021,355,1,111739.204137,83013.860484,131752.168588,111727.925981,111755.098867,...,0.000000,0.000000,0.0,0.0,0.0,106885.837265,1,102887.905090,-0.6,79.0
930,51,12,2021,356,2,111838.001043,76134.289971,125455.782296,111823.380469,111857.634398,...,0.000000,0.000000,0.0,0.0,0.0,100492.746190,1,86748.382460,-1.4,73.0
931,51,12,2021,357,3,111936.797950,68670.874377,118357.290890,111919.085168,111960.086971,...,0.000000,0.000000,0.0,0.0,0.0,93632.999568,1,74357.172142,-2.0,69.0
932,51,12,2021,358,4,112035.594856,56591.338534,103207.709160,112014.024357,112064.214262,...,-0.088575,-0.088575,0.0,0.0,0.0,79657.624656,0,74029.801498,5.7,82.0


In [507]:
model.feature_importances_

array([1.8615745e-03, 3.0788768e-03, 5.5951055e-04, 1.9461932e-03,
       8.5148485e-03, 3.3773404e-02, 2.5219111e-02, 6.2284142e-02,
       0.0000000e+00, 0.0000000e+00, 8.3996123e-03, 0.0000000e+00,
       0.0000000e+00, 6.0137091e-07, 0.0000000e+00, 0.0000000e+00,
       1.8486766e-04, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 3.2331920e-07, 0.0000000e+00,
       0.0000000e+00, 2.4289748e-09, 0.0000000e+00, 0.0000000e+00,
       2.0981652e-03, 0.0000000e+00, 0.0000000e+00, 1.5956929e-03,
       0.0000000e+00, 0.0000000e+00, 1.3698188e-06, 0.0000000e+00,
       0.0000000e+00, 5.3850713e-04, 0.0000000e+00, 0.0000000e+00,
       1.1288516e-02, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 5.9439964e-03, 0.0000000e+00, 0.0000000e+00,
       2.9241270e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 2.2332244e-02, 0.0000000e

In [583]:
results = pd.DataFrame()
results['actual'] = y_test
results['prediction'] = y_pred
results['prophet'] = data['yhat'][-30:]
results

,actual,prediction,prophet
916,96196.124898,83810.773438,99036.496545
917,92744.263348,67658.492188,91180.400508
918,101228.700023,141855.796875,117891.200706
919,107064.467839,132872.703125,124649.179829
920,110698.874419,88844.664062,117475.785954
921,100503.117963,89580.734375,115301.625990
922,97576.039846,98459.632812,112108.702382


In [619]:
param_grid={"learning_rate": (0.05, 0.10, 0.15, 0.2),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[0.2, 0.3, 0.4],}

In [620]:
gs = GridSearchCV(estimator=xgb.XGBRegressor(), param_grid=param_grid,
            cv=3, scoring='neg_mean_squared_error',  n_jobs=-1)

In [621]:
gs.fit(X_train,y_train)


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                              

In [622]:
gs.best_params_

{'colsample_bytree': 0.3,
 'gamma': 0.0,
 'learning_rate': 0.15,
 'max_depth': 5,
 'min_child_weight': 1}